In [1]:
import pandas as pd
from haversine import haversine, Unit

In [2]:
lm_df = pd.read_csv("landmarks_manual.csv")

In [3]:
lm_df

,landmark,station,dir,lat,long,x_pixel,y_pixel,x_resolution,resolution_y
0,69braco helipad,69bravo,n,34.075063,-118.627506,2385,1343,3072,2048
1,69bravo circle building,69bravo,n,34.078233,-118.628952,933,1061,3072,2048
2,cylinder building - close,69bravo,e,34.074371,-118.626666,1741,1613,3072,2048
3,cylinder building - middle,69bravo,e,34.074346,-118.625853,1600,1470,3072,2048
4,cylinder building - far,69bravo,e,34.074621,-118.625100,1371,1433,3072,2048
...,...,...,...,...,...,...,...,...,...
110,building - solar panels,ws,w,32.647416,-116.320512,1583,1366,3072,2048
111,north building,ws,w,32.648401,-116.319985,2470,1412,3072,2048
112,storage silo,ws,w,32.646936,-116.319874,1110,1430,3072,2048
113,building off road,ws,n,32.650992,-116.316876,1947,1192,3072,2048


In [4]:
cam_meta = pd.read_csv("camera_metadata.csv")
cam_meta['lat_adjusted'] = cam_meta['lat_adjusted'].fillna(cam_meta['lat'])
cam_meta['long_adjusted'] = cam_meta['long_adjusted'].fillna(cam_meta['long'])
cam_meta[['code','dir']] = cam_meta['camera_name'].str.split("-", expand=True)[[0,1]]

In [5]:
cam_meta

,camera_id,lat,lat_decimals,long,camera_name,hpweren_camera_description,lat_adjusted,long_adjusted,sample_image_url,camera_model,comment,code,dir
0,hpwren0_unknown direction,32.550000,2,-117.120000,tje-1-mobo-c,NaN,32.550000,-117.120000,NaN,NaN,NaN,tje,1
1,hpwren1_north,33.159900,4,-116.808100,bm-n-mobo-c,Big Black Mountain,33.159900,-116.808100,NaN,NaN,NaN,bm,n
2,hpwren1_east,33.159900,4,-116.808100,bm-e-mobo-c,Big Black Mountain,33.159900,-116.808100,NaN,NaN,NaN,bm,e
3,hpwren1_south,33.159900,4,-116.808100,bm-s-mobo-c,Big Black Mountain,33.159900,-116.808100,NaN,NaN,NaN,bm,s
4,hpwren1_west,33.159900,4,-116.808100,bm-w-mobo-c,Big Black Mountain,33.159900,-116.808100,NaN,NaN,NaN,bm,w
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170,unknown,33.711172,4,-117.534115,stgo-w-mobo-c,Santiago Peak,33.711172,-117.534115,NaN,NaN,NaN,stgo,w
1171,unknown,32.647266,4,-116.318014,ws-n-mobo-c,White Star,32.647266,-116.318014,NaN,NaN,NaN,ws,n
1172,unknown,32.647266,4,-116.318014,ws-e-mobo-c,White Star,32.647266,-116.318014,NaN,NaN,NaN,ws,e
1173,unknown,32.647266,4,-116.318014,ws-s-mobo-c,White Star,32.647266,-116.318014,NaN,NaN,NaN,ws,s


In [6]:
lm_df = lm_df.merge(cam_meta[['code', 'dir', 'lat_adjusted', 'long_adjusted']], left_on=['station', 'dir'], right_on=['code', 'dir'], how='left')

In [7]:
lm_df.head()

,landmark,station,dir,lat,long,x_pixel,y_pixel,x_resolution,resolution_y,code,lat_adjusted,long_adjusted
0,69braco helipad,69bravo,n,34.075063,-118.627506,2385,1343,3072,2048,69bravo,34.074585,-118.627884
1,69braco helipad,69bravo,n,34.075063,-118.627506,2385,1343,3072,2048,69bravo,34.074585,-118.627884
2,69bravo circle building,69bravo,n,34.078233,-118.628952,933,1061,3072,2048,69bravo,34.074585,-118.627884
3,69bravo circle building,69bravo,n,34.078233,-118.628952,933,1061,3072,2048,69bravo,34.074585,-118.627884
4,cylinder building - close,69bravo,e,34.074371,-118.626666,1741,1613,3072,2048,69bravo,34.074585,-118.627884


In [8]:
lm_df = lm_df.drop_duplicates().reset_index(drop=True)

In [9]:
lm_df['distance'] = lm_df.apply(lambda x: haversine((x['lat'], x['long']), (x['lat_adjusted'], x['long_adjusted'])), axis=1)
del lm_df['code']
del lm_df['lat_adjusted']
del lm_df['long_adjusted']

In [10]:
lm_df

,landmark,station,dir,lat,long,x_pixel,y_pixel,x_resolution,resolution_y,distance
0,69braco helipad,69bravo,n,34.075063,-118.627506,2385,1343,3072,2048,0.063468
1,69bravo circle building,69bravo,n,34.078233,-118.628952,933,1061,3072,2048,0.417427
2,cylinder building - close,69bravo,e,34.074371,-118.626666,1741,1613,3072,2048,0.114662
3,cylinder building - middle,69bravo,e,34.074346,-118.625853,1600,1470,3072,2048,0.188961
4,cylinder building - far,69bravo,e,34.074621,-118.625100,1371,1433,3072,2048,0.256448
...,...,...,...,...,...,...,...,...,...,...
110,building - solar panels,ws,w,32.647416,-116.320512,1583,1366,3072,2048,0.234453
111,north building,ws,w,32.648401,-116.319985,2470,1412,3072,2048,0.223537
112,storage silo,ws,w,32.646936,-116.319874,1110,1430,3072,2048,0.177974
113,building off road,ws,n,32.650992,-116.316876,1947,1192,3072,2048,0.427791


In [11]:
lm_df.to_csv("landmarks_processed.csv", index=False)